# Dense Sentiment Classifier

#### Zaprojektujmy gęstą sieć do klasyfikacji sentymentu (nastroju / nastawienia) recenzji filmów z portalu IMDB

#### Załadujmy zależności (zwróć uwagę na nowy rodzaj warstwy - Embedding)

In [ ]:
import tensorflow
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import ModelCheckpoint 
import os
from sklearn.metrics import roc_auc_score, roc_curve, auc, RocCurveDisplay
import pandas as pd
import matplotlib.pyplot as plt

**Ustawmy stałe i hiperparametry (gdy wiemy, że przetwarzanie zbioru danych będzie bardziej złożone, warto to robić już na początku), wybierz dowolną liczbę epok i batch_size. Przypatrz się co oznaczają wszystkie hiperparametry, przyda się to w zrozumieniu całego procesu przetwarzania**

In [ ]:
output_dir = 'output' #sieć będzie generować wyjście, więc określamy jego lokalizację

#hiperparametry treningu
epochs = ...
batch_size = ...

# hiperparametry przetwarzania tekstu: 
n_dim = 64 #liczba wymiarów przestrzeni wektorów słów
n_unique_words = 5000 # liczba uwzględnianych najpopularniejszych słów w korpusie recenzji filmów
n_words_to_skip = 50 # liczba najczęstszych stop words do usunięcia
max_review_length = 100 # długość recenzji, dłuższe będą przycinane, a krótsze sztucznie wydłużane (tak jakby padding) 
pad_type = trunc_type = 'pre' #przycinanie i wydłużanie będzie miało miejsce na początku recenzji (alternatywa do 'post')

# hiperparametry sieci: 
n_dense = 64
dropout = 0.5

#### Załadujmy dane

Zbiór składa się z 50 000 recenzji - połowa w zbiorze treningowym, połowa w walidacyjnym. Wraz z recenzją dodawane są liczby gwiazdek jako oceną od 1 do 10. Recenzję ocenioną na <=4 gwiazdki uznaje się za negatywną (y=0), a na >=7 gwiazdek za pozytywną (y=1), neutralnych recenzji nie ma w zbiorze.

W metodzie `load_data` kryje się już kilka metod automatycznego przetwarzania tekstu:
 - tokenizacja
 - usuwanie znaków interpunkcyjnych
 - zamiana na małe litery
 - zamiana słów na całkowitoliczbowe indeksy
 - ograniczenie wielkości słownika (num_words)
 - usunięcie najpopularniejszych stop words (skip_top)
 - brakuje w niej ewentualnego stemmingu i lematyzacji oraz analizy n-gramów

In [ ]:
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words, skip_top=n_words_to_skip) 

#### Wyświetlmy pierwsze recenzje ze zbioru treningowego

Tokeny są reprezentowane przez całkowitoliczbowe indeksy, posortowane wg częstości występowania. Pierwsze kilka liczb ma specjalne znaczenie:
 - 0 - token dopełniający
 - 1 - token startowy (początek recenzji), ale tutaj jest to token nieznany gdyż token startowy znajduje się wśród 50 najpopularniejszych tokenów
 - 2 - token występujący bardzo często (czyli usunięty jako stop word) lub bardzo rzadko (czyli nieuwzględniony), zastąpiony w związku z tym tokenem 'nieznany' (UNK)
 - 3 - słowo które występuje w korpusie najczęściej
 - 4 - drugie najczęściej występujące słowo
 - 5 - trzecie najczęściej występujące słowo itd.

In [ ]:
x_train[0:6]

#### Wyświetlmy długość pierwszych recenzji (liczbę tokenów w nich) oraz ich sentyment

In [ ]:
for x in x_train[0:6]:
    print(len(x))

In [ ]:
y_train[0:6]

#### Sprawdźmy jak de facto wyglądają słowa, kryjące się pod indeksami

In [ ]:
word_index = tensorflow.keras.datasets.imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["PAD"] = 0
word_index["START"] = 1
word_index["UNK"] = 2

In [ ]:
word_index

**Zadanie 0. Stwórz odwrotność powyższego słownika, by móc odczytywać recenzje**

In [ ]:
index_word = #tu odpowiedź

#### Odczytajmy pierwszą z recenzji, indeksowo, a potem słownie

In [ ]:
x_train[0]

In [ ]:
' '.join(index_word[id] for id in x_train[0])

#### Porównajmy sobie to z oryginalną recenzją. W tym celu załadujemy pełną bazę danych recenzji

In [ ]:
(all_x_train,_),(all_x_valid,_) = imdb.load_data() 

In [ ]:
' '.join(index_word[id] for id in all_x_train[0])

#### Przetwórzmy dane poprzez ujednolicenie wielkości danych wejściowych (dopełnienie albo przycięcie)

In [ ]:
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

In [ ]:
x_train[0:6]

#### Sprawdźmy czy to się udało i jak w praktyce wygląda to ujednolicenie

In [ ]:
for x in x_train[0:6]:
    print(len(x))

In [ ]:
' '.join(index_word[id] for id in x_train[0])

In [ ]:
' '.join(index_word[id] for id in x_train[5])

#### Zaprojektujmy sieć jednokierunkową do przetworzenia recenzji

**Zadanie 1.  Zaprojektuj sieć o następujących warstwach:**

    - embedding, z liczbą unikalnych słów, liczbą wymiarów przestrzeni wektorów i maksymalną długością recenzji jako parametrami (sprawdź w dokumentacji tf)
    - warstwa spłaszczająca
    - dwie warstwy gęste ReLU z dodanymi odpowiednio dropoutami
    - warstwę klasyfikującą recenzje na pozytywną i negatywną

In [ ]:
#tu odpowiedź

In [ ]:
model.summary()

**Zadanie 2. Z czego wynikają liczby parametrów warstwy embedding i pierwszej warstwy gęstej?**

tu odpowiedź

#### Skonfigurujmy model

**Zadanie 3. Skompiluj model z odpowiednimi dla tego problemu funkcją kosztu i metryką oraz dowolnym optymalizatorem**

In [ ]:
#tu odpowiedź

#### Stwórzmy obiekt i katalog do rejestrowania wag modelu

In [ ]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

**Zadanie 4. Naucz sieć na zbiorze treningowym z wyznaczonymi na wstępie hiperparametrami, ze zbiorem walidacyjnym, z użyciem rejestratora wag (sprawdź w dokumentacji tf)**

In [ ]:
#tu odpowiedź

**Zadanie 5. Załaduj wagi z ostatniej epoki (sprawdź w dokumentacji tf) i dokonaj ewaluacji (inferecji) modelu na zbiorze walidacyjnym**

In [ ]:
# tu załaduj wagi

In [ ]:
y_pred = ...

In [ ]:
len(y_pred)

In [ ]:
y_pred[0]

In [ ]:
y_valid[0]

#### Wyświetlmy histogram dla danych walidacyjnych

In [ ]:
plt.hist(y_pred)
_ = plt.axvline(x=0.5, color='orange')

Co widzimy? Model jest najczęściej bardzo zdecydowany, większość wartości znajduje się w skrajnych częściach wykresu. Pomarańczowa linia oznacza próg 0.5, powyżej którego model po prostu ocenia recenzję jako pozytywną. Spróbujmy jeszcze zatem na różne sposoby zbadać skuteczność klasyfikacji binarnej.

In [ ]:
roc_auc_score(y_valid, y_pred)*100.0

In [ ]:
fpr, tpr, thresholds = roc_curve(y_valid, y_pred)
roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Model')
display.plot()
plt.show()

Krzywa roc wizualizuje zależność między skutecznością klasyfikacją próbek pozytywnych a nietrafnością klasyfikacji przypadków negatywnych (np. podział klientów banku na spłacających i niespłacających kredyty) 

#### Zgromadźmy wyniki, np. w postaci DataFrame'a,

In [ ]:
float_y_pred = []
for y in y_pred:
    float_y_pred.append(y[0])
ydf = pd.DataFrame(list(zip(float_y_pred, y_valid)), columns=['y_pred', 'y'])
ydf.head(10)

**Zadanie 6. Wyświetl tę recenzję, która została błędnie zaklasyfikowana z największą nieprawidłową pewnością modelu (zauważ i weź pod uwagę, że jeśli wartość prawdopodobieństwa jest bardzo mała, to model również ma dużą pewność - że recenzja jest negatywna)**

In [ ]:
#tu odpowiedź